In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import median_absolute_error, make_scorer

from sklearn.model_selection import cross_validate, KFold
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler

from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor, ExtraTreesRegressor, GradientBoostingRegressor, RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.linear_model import RidgeCV, PassiveAggressiveRegressor, LinearRegression, TheilSenRegressor, HuberRegressor, RANSACRegressor, Lasso, ElasticNet, Lars, LassoLars, OrthogonalMatchingPursuit, BayesianRidge, ARDRegression, TweedieRegressor, PoissonRegressor, GammaRegressor, LassoLars
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

experiment = 'robustscaler_no_keras'

import warnings
warnings.filterwarnings("ignore")

c:\Users\Adeniyi Babalola\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [2]:
train = pd.read_csv('train.csv')
artificial = pd.read_csv('Artificial_Crystals_Dataset.csv')
mineral = pd.read_csv('Mineral_Dataset_Supplementary_Info.csv')
test = pd.read_csv('test.csv')

train.head()

,id,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average,Hardness
0,0,100.0,0.841611,10.0,4.8,20.612526,11.08810,2.766,1.732,0.860,0.496070,0.91457,6.0
1,1,100.0,7.558488,10.0,4.8,20.298893,12.04083,2.755,1.631,0.910,0.492719,0.71760,6.5
2,2,76.0,8.885992,15.6,5.6,33.739258,12.08630,2.828,1.788,0.864,0.481478,1.50633,2.5
3,3,100.0,8.795296,10.0,4.8,20.213349,10.94850,2.648,1.626,0.936,0.489272,0.78937,6.0
4,4,116.0,9.577996,11.6,4.8,24.988133,11.82448,2.766,1.682,0.896,0.492736,1.86481,6.0


In [3]:
# Rename hardness in artifical dataset and drop columns not required
artificial.rename(columns={'Hardness (Mohs)': 'Hardness'}, inplace=True)
artificial.drop(['Formula', 'Crystal structure', 'Unnamed: 0'], axis=1, inplace=True)

artificial.head()

,Hardness,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average
0,4.5,167.0,23.907992,18.555556,5.000000,41.609136,11.693844,2.938889,1.711111,0.884444,0.477830,2.656444
1,4.0,14.0,1.740168,4.666667,1.333333,8.773227,11.614333,1.903333,1.310000,0.680000,0.825990,0.580056
2,2.5,102.0,8.511159,4.434783,3.304348,8.440584,13.176622,2.672609,1.379130,0.530870,0.713850,0.370050
3,5.5,78.0,8.109328,13.000000,5.333333,27.448814,11.826400,2.960000,1.625000,0.813333,0.488163,1.351555
4,6.5,164.0,19.921324,14.909091,5.090909,32.012361,11.255573,2.881818,1.640909,0.841818,0.483480,1.811029


In [4]:
# Drop unneeded columns
mineral.drop('Unnamed: 0', axis=1, inplace=True)

mineral.head()

,Hardness,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average
0,2.3,110.0,23.000000,36.666667,2.666667,82.598467,8.504133,2.146667,2.006667,1.253333,0.456803,7.666667
1,5.5,406.0,30.472136,9.902439,4.682927,19.813180,11.456151,2.700244,1.676829,0.868293,0.522909,0.743223
2,5.5,406.0,30.472464,10.410256,4.923077,20.931371,11.541405,2.753590,1.703846,0.894359,0.497498,0.781345
3,5.5,476.0,61.142136,11.609756,4.682927,23.659644,11.487395,2.763659,1.714634,0.848780,0.519474,1.491272
4,5.5,476.0,61.142464,12.205128,4.923077,24.975089,11.574251,2.820256,1.743590,0.873846,0.493887,1.567755


In [5]:
train.shape, artificial.shape, mineral.shape

((10407, 13), (52, 12), (622, 12))

In [6]:
# train = pd.concat([train, artificial, mineral], axis=0)
# train.shape

In [7]:
# Constants
TARGET = 'Hardness'

In [8]:
# Selecting numerical variables from the train dataset, excluding 'id' and TARGET
num_var = train.drop(['id', TARGET], axis=1).select_dtypes(include=np.number).columns
print(num_var)

Index(['allelectrons_Total', 'density_Total', 'allelectrons_Average',
       'val_e_Average', 'atomicweight_Average', 'ionenergy_Average',
       'el_neg_chi_Average', 'R_vdw_element_Average', 'R_cov_element_Average',
       'zaratio_Average', 'density_Average'],
      dtype='object')


In [9]:
len(num_var)

11

In [10]:
# # Combining train and test datasets for comparative analysis
# # 'Source' column is added to label data from each dataset
# df = pd.concat([
#     train[num_var].assign(Source='Train'), 
#     test[num_var].assign(Source='Test')
# ], axis=0, ignore_index=True)

In [11]:
# # Check for outliers
# def outlier_thresholds(dataframe, col_name, q1=0.01, q3=0.99):
#     quartile1 = dataframe[col_name].quantile(q1)
#     quartile3 = dataframe[col_name].quantile(q3)
#     IQR = quartile3 - quartile1
#     up_limit = quartile3 + 1.5 * IQR
#     low_limit = quartile1 - 1.5 * IQR
#     print(f'Feature name: {col_name}')
#     print(f'Low Limit: {low_limit}')
#     print(f'Upper Limit: {up_limit}')
#     print()
#     return low_limit, up_limit


# def remove_outlier(dataframe, col_name):
#     """
#     Example Usage:
#     for col in num_cols:
#     new_df = remove_outlier(titanic, col)
#     """
#     low_limit, up_limit = outlier_thresholds(dataframe, col_name)
#     df_without_outliers = dataframe[~((dataframe[col_name] < low_limit) | (dataframe[col_name] > up_limit))]    
#     return df_without_outliers

# def cap_outliers(dataframe, col_name):
#     low_limit, up_limit = outlier_thresholds(dataframe, col_name)

#     dataframe[col_name] = np.where(dataframe[col_name] > up_limit, up_limit, 
#                                    np.where(dataframe[col_name] < low_limit, low_limit, dataframe[col_name]))
#     return dataframe

# df_outliers = train.copy()
# df_outliers_test = test.copy()

# for col in num_var:
#     df_outliers = remove_outlier(df_outliers, col)
#     df_outliers_test = remove_outlier(df_outliers_test, col)

# df_outliers.columns

In [12]:
scaler = MinMaxScaler()

standard_scaler = scaler.fit_transform(train[num_var])
train_scaled = pd.DataFrame(standard_scaler, columns=num_var)

test_scaled = scaler.transform(test[num_var])
test_scaled = pd.DataFrame(test_scaled, columns=num_var)
# X = pd.concat([X[cat_features], train_scaled], axis=1)
train_scaled.head()

,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average
0,0.006536,0.001309,0.149254,0.800000,0.123133,0.727288,0.803369,0.769778,0.532231,0.600576,0.083370
1,0.006536,0.011753,0.149254,0.800000,0.121260,0.789780,0.800174,0.724889,0.563175,0.596519,0.065415
2,0.004967,0.013818,0.232836,0.933333,0.201549,0.792762,0.821377,0.794667,0.534706,0.582910,0.137314
3,0.006536,0.013677,0.149254,0.800000,0.120749,0.718132,0.769097,0.722667,0.579265,0.592346,0.071957
4,0.007582,0.014894,0.173134,0.800000,0.149272,0.775589,0.803369,0.747556,0.554510,0.596540,0.169992


In [13]:
# test_scaled.head()

# MODEL CROSS-VALIDATION

In [14]:
MLA = [
	# Trial Models
	MLPRegressor(random_state=5),
	TheilSenRegressor(random_state=5),
	HuberRegressor(),
	RANSACRegressor(random_state=5),
	Lasso(random_state=5),
	ElasticNet(random_state=5),
	Lars(random_state=5),
	LassoLars(random_state=5),
	OrthogonalMatchingPursuit(),
	BayesianRidge(),
	ARDRegression(),
    TweedieRegressor(power=1.5, alpha=0.5),
    PoissonRegressor(alpha=0.5),
    GammaRegressor(alpha=0.5),
    LassoLars(alpha=0.1, random_state=5),

	# GLM
	LinearRegression(),
	PassiveAggressiveRegressor(random_state=5),
	RidgeCV(),

	# Trees    
	DecisionTreeRegressor(random_state=5),
	ExtraTreeRegressor(random_state=5),

	XGBRegressor(random_state=5),
	LGBMRegressor(n_jobs=-1, random_state=5),
	CatBoostRegressor(random_state=5, verbose=False, early_stopping_rounds=100),
	
	# KNeighbors
	KNeighborsRegressor(),
	KNeighborsRegressor(n_neighbors=2),
	KNeighborsRegressor(n_neighbors=4),
	KNeighborsRegressor(n_neighbors=8),
	KNeighborsRegressor(n_neighbors=16),
	KNeighborsRegressor(n_neighbors=32),
	KNeighborsRegressor(n_neighbors=64),
	KNeighborsRegressor(n_neighbors=128),
	KNeighborsRegressor(n_neighbors=256),
	KNeighborsRegressor(n_neighbors=512),
	KNeighborsRegressor(n_neighbors=1024),

	# Ensemble Methods
	AdaBoostRegressor(random_state=5),
	BaggingRegressor(random_state=5),
	ExtraTreesRegressor(random_state=5),
	GradientBoostingRegressor(random_state=5),
	HistGradientBoostingRegressor(random_state=5),
	RandomForestRegressor(random_state=5),
    ]


# split dataset in cross-validation with splitter class
# cv_split could KFold, StratifiedKFold or RepeatedKFold depending on the problem
cv_split = KFold(n_splits=10, shuffle=True, random_state=5)
cv_split_trial = KFold(n_splits=3, shuffle=True, random_state=5) # For quick trials

In [15]:
# # create table to compare MLA metrics
# MLA_columns = ['MLA Name', 'MLA Parameters', 'MLA Train Accuracy Mean', 'MLA Test Accuracy Mean', 'MLA Test Accuracy 3*STD', 'MLA Time']
# MLA_compare = pd.DataFrame(columns = MLA_columns)

# # create table to compare MLA predictions
# MLA_predict = {}

# # index through MLA and save performance to table
# row_index = 0
# scoring = median_abs_error_scorer = make_scorer(median_absolute_error, greater_is_better=False)

# for alg in MLA:

# 	# set name and parameters
# 	MLA_name = alg.__class__.__name__

# 	# Add suffix if name already exists
# 	suffix = 1
# 	original_MLA_name = MLA_name
# 	while MLA_compare['MLA Name'].str.contains(MLA_name).any():
# 		MLA_name = f"{original_MLA_name}_{suffix}"
# 		suffix += 1
		
# 	MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
# 	MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())

# 	"""score model with cross validation: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate"""

# 	cv_results = cross_validate(alg, train_scaled, train[TARGET], cv=cv_split, scoring=scoring, return_train_score=True)

# 	# Calculate mean time in seconds
# 	mean_fit_time = cv_results['fit_time'].mean()

# 	# Convert mean time to minutes and seconds
# 	minutes = int(mean_fit_time // 60)
# 	seconds = mean_fit_time % 60

# 	# Format the time and assign it
# 	MLA_compare.loc[row_index, 'MLA Time'] = f"{minutes} min {seconds:.2f} sec"
# 	MLA_compare.loc[row_index, 'MLA Train Accuracy Mean'] = cv_results['train_score'].mean() * -1
# 	MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = cv_results['test_score'].mean() * -1
# 	#if this is a non-bias random sample, then +/-3 standard deviations (std) from the mean, should statistically capture 99.7% of the subsets
# 	MLA_compare.loc[row_index, 'MLA Test Accuracy 3*STD'] = cv_results['test_score'].std()*3   #let's know the worst that can happen!

# 	print(f'Done with {MLA_name}')
# 	row_index+=1

# #print and sort table: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html
# MLA_compare.sort_values(by = ['MLA Test Accuracy Mean'], ascending = True, inplace = True)
# MLA_compare.to_csv(f'{experiment}_results.csv', index=False)
# MLA_compare

In [16]:
# sns.barplot(x='MLA Test Accuracy Mean', y = 'MLA Name', data = MLA_compare, color = 'm')

# #prettify using pyplot: https://matplotlib.org/api/pyplot_api.html
# plt.title('Machine Learning Algorithm Accuracy Score \n')
# plt.xlabel('Accuracy Score (%)')
# plt.ylabel('Algorithm')

# ENSEMBLE CROSS-VALIDATION

In [17]:
# Hill Climbing inspired by code from Kaggle
def hill_climbing(x, y):
    
    # Evaluating oof predictions
    scores = {}
    for col in x.columns:
        scores[col] = median_absolute_error(y, x[col])

    # Sorting the model scores in ascending order
    scores = {k: v for k, v in sorted(scores.items(), key = lambda item: item[1], reverse = False)}

    # Sort oof_df
    x = x[list(scores.keys())]

    # Initialize weights
    weights = {col: 1 if i == 0 else 0 for i, col in enumerate(x.columns)}

    STOP = False
    current_best_ensemble = x.iloc[:,0]
    MODELS = x.iloc[:,1:]
    weight_range = np.arange(-0.5, 0.51, 0.01) 
    history = [median_absolute_error(y, current_best_ensemble)]
    j = 0

    while not STOP:
        j += 1
        potential_new_best_cv_score = median_absolute_error(y, current_best_ensemble)
        k_best, wgt_best = None, None
        for k in MODELS:
            for wgt in weight_range:
                potential_ensemble = (1 - wgt) * current_best_ensemble + wgt * MODELS[k]
                cv_score = median_absolute_error(y, potential_ensemble)
                if cv_score < potential_new_best_cv_score:
                    potential_new_best_cv_score = cv_score
                    k_best, wgt_best = k, wgt

        if k_best is not None:
            # Update weights
            weights = {col: (1 - wgt_best) * weights[col] if col != k_best else wgt_best for col in weights}
            current_best_ensemble = (1 - wgt_best) * current_best_ensemble + wgt_best * MODELS[k_best]
            MODELS.drop(k_best, axis = 1, inplace = True)
            if MODELS.shape[1] == 0:
                STOP = True
            history.append(potential_new_best_cv_score)
        else:
            STOP = True
        
    hill_ens_pred = current_best_ensemble
    
    return hill_ens_pred, weights

In [18]:
def create_submission_file(ids, predictions, filename):
    submission = pd.DataFrame({'id': ids, 'Hardness': predictions})
    submission.to_csv(filename, index=False)

In [19]:
hill_climb_scores = []
hill_climb_weights = []

for i, (train_index, test_index) in enumerate(cv_split.split(train_scaled, train[TARGET])):
    X_train, X_test = train_scaled.iloc[train_index], train_scaled.iloc[test_index]
    y_train, y_test = train[TARGET].iloc[train_index], train[TARGET].iloc[test_index]

    print(f'Fold {i+1}')
    
    MLA_cv_train_preds = []
    MLA_cv_preds = []
    MLA_cv_preds_dict = {}
    MLA_names = []
    
    suffix = 1
    for alg in MLA:
        MLA_name = alg.__class__.__name__

        # Add suffix if name already exists

        original_MLA_name = MLA_name
        if MLA_name in MLA_names:
        # while MLA_cv_preds.str.contains(MLA_name).any():
            MLA_name = f"{original_MLA_name}_{suffix}"
            suffix += 1
            
        predictor = alg.fit(X_train, y_train)
        pred_train_result = predictor.predict(X_train)
        pred_result = predictor.predict(X_test)

        MLA_cv_train_preds.append(pred_train_result)
        MLA_cv_preds.append(pred_result)
        MLA_cv_preds_dict[MLA_name] = pred_result
        MLA_names.append(MLA_name)

    ##################
    ### Hill Climb ###
    ##################
    hill_climb_pred, hill_climb_weight = hill_climbing(pd.DataFrame(MLA_cv_preds_dict), y_test)
    hill_climb_score = median_absolute_error(y_test, hill_climb_pred)
    hill_climb_scores.append(hill_climb_score)
    hill_climb_weights.append(hill_climb_weight)
    print(f'The Fold {i+1} Hill Climb is {hill_climb_score}')
    print(f'The Fold {i+1} weight is {hill_climb_weight}')
    print()

print()
print(f'The Hill Climbing CV score is ==> {np.mean(hill_climb_scores)}')
print(f'The Hill Climbing weights are ==> {hill_climb_weights}')

Fold 1
The Fold 1 Hill Climb is 0.6004305267220316
The Fold 1 weight is {'XGBRegressor': 0.694317761279997, 'RandomForestRegressor': 0.0, 'ExtraTreesRegressor': -0.009999999999999554, 'CatBoostRegressor': 0.010100000000000442, 'LGBMRegressor': 0.0, 'HistGradientBoostingRegressor': 0.0, 'BaggingRegressor': 0.3738634099199997, 'GradientBoostingRegressor': 0.0, 'KNeighborsRegressor_4': 0.0, 'KNeighborsRegressor_6': 0.0, 'KNeighborsRegressor_3': 0.0399960000000004, 'KNeighborsRegressor_5': 4.440892098500624e-16, 'KNeighborsRegressor': 4.440892098500626e-16, 'KNeighborsRegressor_7': 0.0, 'KNeighborsRegressor_8': 0.0, 'DecisionTreeRegressor': 0.0, 'ExtraTreeRegressor': 0.0, 'KNeighborsRegressor_2': -0.06719327999999948, 'KNeighborsRegressor_9': 0.0, 'KNeighborsRegressor_10': 0.0, 'KNeighborsRegressor_11': 0.0, 'MLPRegressor': 0.0, 'HuberRegressor': 0.0, 'TheilSenRegressor': -0.04108389119999946, 'AdaBoostRegressor': 0.0, 'Lars': 0.0, 'LinearRegression': 0.0, 'RidgeCV': 0.0, 'ARDRegression': 

In [20]:
# Get average weights for the models from all the folds

average_values = {}

for model in hill_climb_weights:
    for key, value in model.items():
        if key in average_values:
            average_values[key] += value
        else:
            average_values[key] = value

num_models = len(hill_climb_weights)
average_values = {k: v / num_models for k, v in average_values.items()}

# Ensure the new weights sum up to 1
sum = 0

for k, v in average_values.items():
    sum += v

average_values, sum, len(average_values)

({'XGBRegressor': 0.1338754617279997,
  'RandomForestRegressor': 0.10050589739519852,
  'ExtraTreesRegressor': 0.12371334022659979,
  'CatBoostRegressor': 0.004010000000000213,
  'LGBMRegressor': 0.31613265934999835,
  'HistGradientBoostingRegressor': 0.3841816366349985,
  'BaggingRegressor': 0.05149533399200016,
  'GradientBoostingRegressor': 0.005761649000000241,
  'KNeighborsRegressor_4': 8.881784197001211e-17,
  'KNeighborsRegressor_6': 0.0010000000000001332,
  'KNeighborsRegressor_3': 0.013988313000000117,
  'KNeighborsRegressor_5': 0.004038384000000072,
  'KNeighborsRegressor': -0.0006474749999998816,
  'KNeighborsRegressor_7': -1.9899999999832378e-05,
  'KNeighborsRegressor_8': 0.01163054591999996,
  'DecisionTreeRegressor': 0.024045600000000222,
  'ExtraTreeRegressor': 1.7763568394002424e-16,
  'KNeighborsRegressor_2': -0.00671932799999986,
  'KNeighborsRegressor_9': 1.3322676295501846e-16,
  'KNeighborsRegressor_10': 8.881784197001238e-17,
  'KNeighborsRegressor_11': 1.3322676

In [21]:
# Create an ordered dictionary based on the order of models in MLA
ordered_average_values = {}
final_mlas = []

for model_name in MLA_names:        
    if model_name in average_values:
        ordered_average_values[model_name] = average_values[model_name]
    else:
        # Handle case where a model might not be in average_values
        ordered_average_values[model_name] = None

# Now ordered_average_values has the averages in the same order as MLA
ordered_average_values

{'MLPRegressor': 0.001000000000000134,
 'TheilSenRegressor': -0.004108389119999813,
 'HuberRegressor': 8.881784197001171e-17,
 'RANSACRegressor': 0.008002500000000159,
 'Lasso': -0.03561567207659981,
 'ElasticNet': -0.000989999999999911,
 'Lars': 8.881784197001178e-17,
 'LassoLars': 4.4408920985006005e-17,
 'OrthogonalMatchingPursuit': -0.0019795999999998814,
 'BayesianRidge': 8.881784197001189e-17,
 'ARDRegression': 8.881784197001205e-17,
 'TweedieRegressor': -0.025056163439999703,
 'PoissonRegressor': -0.030768307364999803,
 'GammaRegressor': -0.016205227315199695,
 'LassoLars_1': 4.4408920985006024e-17,
 'LinearRegression': 8.881784197001178e-17,
 'PassiveAggressiveRegressor': 0.0059815800000001754,
 'RidgeCV': 8.881784197001189e-17,
 'DecisionTreeRegressor': 0.024045600000000222,
 'ExtraTreeRegressor': 1.7763568394002424e-16,
 'XGBRegressor': 0.1338754617279997,
 'LGBMRegressor': 0.31613265934999835,
 'CatBoostRegressor': 0.004010000000000213,
 'KNeighborsRegressor': -0.00064747499

In [22]:
# Get the ordered weights values as a list to be used for final submission
hill_climb_final_weights = []

for value in ordered_average_values.values():
    hill_climb_final_weights.append(value)

hill_climb_final_weights

[0.001000000000000134,
 -0.004108389119999813,
 8.881784197001171e-17,
 0.008002500000000159,
 -0.03561567207659981,
 -0.000989999999999911,
 8.881784197001178e-17,
 4.4408920985006005e-17,
 -0.0019795999999998814,
 8.881784197001189e-17,
 8.881784197001205e-17,
 -0.025056163439999703,
 -0.030768307364999803,
 -0.016205227315199695,
 4.4408920985006024e-17,
 8.881784197001178e-17,
 0.0059815800000001754,
 8.881784197001189e-17,
 0.024045600000000222,
 1.7763568394002424e-16,
 0.1338754617279997,
 0.31613265934999835,
 0.004010000000000213,
 -0.0006474749999998816,
 -0.00671932799999986,
 0.013988313000000117,
 8.881784197001211e-17,
 0.004038384000000072,
 0.0010000000000001332,
 -1.9899999999832378e-05,
 0.01163054591999996,
 1.3322676295501846e-16,
 8.881784197001238e-17,
 1.3322676295501858e-16,
 -0.06725283892999943,
 0.05149533399200016,
 0.12371334022659979,
 0.005761649000000241,
 0.3841816366349985,
 0.10050589739519852]

In [23]:
test_predictions = []
# Make predictions on test set
for alg in MLA:
        MLA_name = alg.__class__.__name__
                
        predictor = alg.fit(train_scaled, train[TARGET])
        pred_result = predictor.predict(test_scaled)

        test_predictions.append(pred_result)
        print(f'Done with {MLA_name}')

Done with MLPRegressor
Done with TheilSenRegressor
Done with HuberRegressor
Done with RANSACRegressor
Done with Lasso
Done with ElasticNet
Done with Lars
Done with LassoLars
Done with OrthogonalMatchingPursuit
Done with BayesianRidge
Done with ARDRegression
Done with TweedieRegressor
Done with PoissonRegressor
Done with GammaRegressor
Done with LassoLars
Done with LinearRegression
Done with PassiveAggressiveRegressor
Done with RidgeCV
Done with DecisionTreeRegressor
Done with ExtraTreeRegressor
Done with XGBRegressor
Done with LGBMRegressor
Done with CatBoostRegressor
Done with KNeighborsRegressor
Done with KNeighborsRegressor
Done with KNeighborsRegressor
Done with KNeighborsRegressor
Done with KNeighborsRegressor
Done with KNeighborsRegressor
Done with KNeighborsRegressor
Done with KNeighborsRegressor
Done with KNeighborsRegressor
Done with KNeighborsRegressor
Done with KNeighborsRegressor
Done with AdaBoostRegressor
Done with BaggingRegressor
Done with ExtraTreesRegressor
Done with 

In [24]:
# Double check that the weights and predictions are the same length
len(test_predictions), len(hill_climb_final_weights)

(40, 40)

In [25]:
weighted_avg_predictions = np.average(test_predictions, axis=0, weights=hill_climb_final_weights)
create_submission_file(test['id'], weighted_avg_predictions, f'submission_{experiment}.csv')